In [1]:
# Dataset curation notebook

In [14]:
from rdkit import Chem
from rdkit.Chem import AllChem
from ase.calculators.mopac import MOPAC
from ase.io import read
import pandas as pd
import numpy as np
import time

In [33]:
dataset = pd.read_csv('../../Data/Solubility/dataset-C.csv')

In [34]:
#SMILES
smiles = dataset.columns[4]
smiles_list = np.array(dataset[smiles])
print (smiles_list)
#ID
ID = dataset.columns[0]
id_list = np.array(dataset[ID])
print (id_list)

#initialize lists
failures = []
pse_list = []

['CC(C)(C)CO' 'CN=C=S' 'C1OCOCO1' ...
 'CCC(C)C1NC(=O)C(CC2=C[NH]C3=C2C=CC=C3)NC(=O)C4CCCN4C(=O)C(CC5=CC=CC=C5)N(C)C(=O)C6CNCCN6C(=O)C7CCCCN7C1=O'
 'CCC(C)C1NC(=O)C(CC2=C[NH]C3=C2C=CC=C3)NC(=O)C4CCCN4C(=O)C(CC5=C[NH]C=N5)NC(=O)C6CCCCN6C(=O)C7CCC=NN7C1=O'
 'CCC(C)C1NC(=O)C(CC2=CC=CC=C2)NC(=O)C3CCCN3C(=O)C(CC4=CC=CC=C4)N(C)C(=O)C5CCC=NN5C(=O)C6CCC=NN6C1=O']
['C-1' 'C-2' 'C-3' ... 'C-2601' 'C-2602' 'C-2603']


In [35]:
def generate():
    
    start = time.time()
    print ('starting...')
    for i,smiles in enumerate(smiles_list):
        try:
    
            #create mol object
            mol = Chem.MolFromSmiles(smiles)
            mol = Chem.AddHs(mol)

            #embed molecule
            AllChem.EmbedMolecule(mol)

            #create xyz file
            fileName = 'xyz/'+id_list[i] + '.xyz'
            Chem.MolToXYZFile(mol,fileName)

            #use ASE to read xyz file
            mol = read(fileName)

            #calculate 
            mol.calc = MOPAC(label='TMP', task = 'UHF BONDS GRADS')

            #get potential energy
            pse = mol.get_potential_energy()

            #save into list
            pse_list.append(pse)
            
            print (id_list[i], ' done')
        except:
            print ('error in ', id_list[i])
            failures.append(smiles)
            continue

        
            
    print ('DONE. time taken = ', time.time()- start)

In [ ]:
generate()

starting...
C-1  done
C-2  done
C-3  done
C-4  done
C-5  done
C-6  done
C-7  done
C-8  done
C-9  done
C-10  done
C-11  done
C-12  done
C-13  done
C-14  done
C-15  done
C-16  done
C-17  done
C-18  done
C-19  done
C-20  done
C-21  done
C-22  done
C-23  done
C-24  done
C-25  done
C-26  done
C-27  done
C-28  done
error in  C-29
C-30  done
error in  C-31
C-32  done
error in  C-33
C-34  done
error in  C-35
C-36  done
C-37  done
error in  C-38
C-39  done
C-40  done
C-41  done
error in  C-42
error in  C-43
C-44  done
C-45  done
error in  C-46
error in  C-47
error in  C-48
error in  C-49
C-50  done
C-51  done
C-52  done
C-53  done
error in  C-54
error in  C-55
C-56  done
error in  C-57
error in  C-58
C-59  done
error in  C-60
error in  C-61
error in  C-62
error in  C-63
error in  C-64
C-65  done
error in  C-66
C-67  done
C-68  done
C-69  done
C-70  done
error in  C-71
C-72  done
error in  C-73
C-74  done
C-75  done
C-76  done
C-77  done
error in  C-78
C-79  done
C-80  done
C-81  done
C-82  done

In [ ]:
# remove molecules that did not work

In [ ]:
id_to_remove = []
for i, smile in enumerate (smiles_list):
    if smile in failures:
        id_to_remove.append(i)

new_smiles_list = np.delete(smiles_list, id_to_remove)      

In [42]:
#double check list lengths
print(len(pse_list))
print(len(new_smiles_list))

2330
2330


In [ ]:
# convert to dataframe
def make_dataframe(new_smiles_list, pse_list):
    descriptor_data = { 'Smiles': new_smiles_list, 'Energy':pse_list}
    df = pd.DataFrame(data = descriptor_data)
    df.to_csv('../../Data/Energy/EnergyDataset-C.csv')
    print ('shape: ', df.shape)

In [ ]:
make_dataframe(new_smiles_list, pse_list)